### before
├── images
│   └── test_image.png
│   └── test_image2.png
├── maskGen_json.json
└── maskGen.py

### after
├── images
│   └── test_image.png
│   └── test_image2.png
├── masks
│   └── test_image
│         └── test_image-1.png
│         └── test_image-2.png
│   └── test_image2
│         └── test_image2-1.png
│         └── test_image2-2.png
├── maskGen_json.json
└── maskGen.py

In [61]:
import os
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np

def create_seperate_mask_img(json_file_name) :
    # 현재 디렉토리에서 가능!
    # 위의 before 구조에 맞게 저장되어있어야함.
    ## 구조는 마크다운 셀 더블클릭해야 제대로 보임
    source_folder = os.path.join(os.getcwd(), "train_images")
    json_path = json_file_name                     # Relative to root directory
    count = 0                                           # Count of total images saved
    file_bbs = {}                                       # Dictionary containing polygon coordinates for mask
    file_names_dict = {}
    file_names = []
    # Read JSON file
    with open(json_path) as f:
      data = json.load(f)

    # Extract X and Y coordinates if available and update dictionary
    # json 딕셔너리에서 x, y 좌표 추출
    def add_to_dict(data, itr, key, count, file_name):
        # data : json, itr : 이미지 이름, key : mask 이름, count : mask index
        try:
            x_points = data[itr]["regions"][str(count)]["shape_attributes"]["all_points_x"]
            y_points = data[itr]["regions"][str(count)]["shape_attributes"]["all_points_y"]
        except:
            print("No BB. Skipping", key)
            return

        all_points = []
        for i, x in enumerate(x_points):
            all_points.append([x, y_points[i]])

        file_bbs[key] = all_points
        file_names_dict[key] = file_name
        # file_bbs = {image이름-mask이름 : [x points, y points]}

    ## Json 파일에서 x,y points를 뽑아 file_bbs 에 저장.
    for itr in data:
        file_name_json = data[itr]["filename"]
        file_names.append(file_name_json)
        sub_count = 0
        # Contains count of masks for a single ground truth image

        if len(data[itr]["regions"]) > 1: # 마스크의 개수가 여러개
            for _ in range(len(data[itr]["regions"])):
                key = file_name_json[:-4] + "-" + str(sub_count+1)
                add_to_dict(data, itr, key, sub_count, file_name_json)
                sub_count += 1
        else: # 마스크 개수가 1개
            add_to_dict(data, itr, file_name_json[:-4], 0, file_name_json)

    print("\nDict size: ", len(file_bbs))

    ########## 경로 생성
    mask_folder = os.path.join(os.getcwd(), "masks")
    try :
        os.mkdir(mask_folder)
    except :
        print('mask folder already exist')
    for f in file_names :
        try :
            os.mkdir(os.path.join(mask_folder, f[:-4]))
        except :
            print('folder already exist')

    ######### 마스크 저장
    for itr in file_bbs:
        num_masks = itr.split("-")
        save_directory = os.path.join(mask_folder, file_names_dict[itr][:-4])
         # Dimensions should match those of ground truth image
        img = cv2.imread(source_folder+'/'+file_names_dict[itr])
        MASK_WIDTH = img.shape[0]
        MASK_HEIGHT =  img.shape[1]
        mask = np.zeros((MASK_WIDTH, MASK_HEIGHT))
        try:
            arr = np.array(file_bbs[itr])
        except:
            print("Not found:", itr)
            continue
        count += 1

        cv2.fillPoly(mask, [arr], color=(255))

        if len(num_masks) > 1:
            cv2.imwrite(os.path.join(save_directory, itr + ".png") , mask)
        else:
            cv2.imwrite(os.path.join(save_directory, itr + ".png") , mask)

    print("Images saved:", count)

In [17]:
# 마스크들 하나로 합치기
import os
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
import shutil

def merge_masks() :
    # 현재 경로에서 실행
    source_folder = os.path.join(os.getcwd(), "masks")
    mask_folder_list = os.listdir(source_folder)

    for case in mask_folder_list :
        curr_directory = os.path.join(source_folder, case)
        mask = None
        try :
            for i, img in enumerate(os.listdir(curr_directory)) :
                img_directory = os.path.join(curr_directory, img)
                if i == 0 :
                    mask = plt.imread(img_directory)
                else :
                    other_mask = plt.imread(img_directory)
                    mask = np.logical_or(mask, other_mask)
            plt.imsave(os.path.join(source_folder, case+'-mask.png'), mask)
            print('Image Saved')
            if os.path.exists(curr_directory):
                shutil.rmtree(curr_directory)
        except :
            print('Mask Image Not Found')

In [62]:
create_seperate_mask_img('via_region_data.json')

No BB. Skipping CASE14_01-1
No BB. Skipping CASE14_01-2
No BB. Skipping CASE14_01-3
No BB. Skipping CASE14_01-4
No BB. Skipping CASE14_01-5

Dict size:  6
Images saved: 6


In [18]:
merge_masks()

Image Saved
Mask Image Not Found
Image Saved
Mask Image Not Found
Mask Image Not Found
